In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
dataset = pd.read_csv("Data_google_user_reviews.csv")

In [3]:
dataset.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [4]:
dataset = dataset.dropna(axis=0,how='any')

In [5]:
dataset = dataset.head(2000)

In [6]:
pos_count = 0
neg_count = 0
neut_count = 0
nan_values = 0
for i in range(len(dataset)):
    if dataset['Sentiment'].iloc[i] == 'Positive':
        pos_count+=1
    elif dataset['Sentiment'].iloc[i] == 'Negative':
        neg_count+=1
    elif dataset['Sentiment'].iloc[i] == 'Neutral':
        neut_count+=1
    else:
        nan_values+=1

In [7]:
print(pos_count,neg_count,neut_count,nan_values)

1337 366 297 0


In [8]:
labels = np.zeros((len(dataset),1))

In [9]:
for i in range(len(dataset)):
    if dataset['Sentiment'].iloc[i] == 'Positive':
        labels[i][0] = 1
    elif dataset['Sentiment'].iloc[i] == 'Neutral':
        labels[i][0] = 2
    else:
        labels[i][0] = 0

In [10]:
print(labels[:50,0])

[1. 1. 1. 1. 1. 1. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1. 2. 1.
 1. 1. 2. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1.
 2. 1.]


In [11]:
dataset_group = dataset.groupby('App')

In [12]:
app_names = set(dataset.iloc[:,0].values)
print(app_names)

{'8fit Workouts & Meal Planner', 'AMC Theatres', 'ABC Preschool Free', '8 Ball Pool', '2GIS: directory & navigator', 'A+ Mobile', '365Scores - Live Scores', 'AD - Nieuws, Sport, Regio & Entertainment', 'A+ Gallery - Photos & Videos', '850 Sports News Digest', 'AP Mobile - Breaking News', 'ABC Kids - Tracing & Phonics', '591房屋交易-香港', '1800 Contacts - Lens Store', '7 Cups: Anxiety & Stress Chat', '2Date Dating App, Love and matching', 'A Word A Day', 'ABC News - US & World News', '4K Wallpapers and Ultra HD Backgrounds', '591房屋交易-租屋、中古屋、新建案、實價登錄、別墅透天、公寓套房、捷運、買房賣房行情、房價房貸查詢', 'ABCmouse.com', 'ANA', 'APE Weather ( Live Forecast)', '3D Live Neon Weed Launcher', '7 Minute Workout', '2ndLine - Second Phone Number', '7 Weeks - Habit & Goal Tracker', '2RedBeans', 'A Call From Santa Claus!', '30 Day Fitness Challenge - Workout at Home', 'APUS Launcher - Theme, Wallpaper, Hide Apps', 'AC - Tips & News for Android™', '2018Emoji Keyboard 😂 Emoticons Lite -sticker&gif', '21-Day Meditation Experience'

In [13]:
count = []
for i in range(len(app_names)):
    posi_count = 0
    negi_count = 0
    neuti_count = 0
    dataset_app = dataset_group.get_group(app_names.pop())
    for value in dataset_app['Sentiment'].values:
        if str(value) == 'Positive':
            posi_count += 1
        elif str(value) == 'Neutral':
            neuti_count += 1
        else:
            negi_count += 1
    count.append([posi_count,neuti_count,negi_count])

In [14]:
print(count)

[[137, 15, 19], [5, 0, 1], [19, 9, 8], [104, 9, 106], [23, 11, 6], [57, 18, 9], [5, 2, 0], [13, 3, 3], [24, 10, 7], [38, 0, 1], [31, 1, 4], [30, 6, 1], [5, 3, 1], [64, 10, 6], [7, 0, 1], [26, 5, 7], [3, 1, 0], [18, 23, 29], [7, 0, 2], [5, 8, 2], [25, 2, 9], [6, 10, 11], [25, 6, 2], [2, 0, 0], [10, 0, 1], [17, 16, 7], [10, 0, 4], [31, 6, 2], [20, 0, 14], [27, 2, 2], [25, 2, 4], [18, 7, 10], [25, 6, 1], [68, 2, 10], [18, 9, 6], [27, 3, 8], [17, 0, 3], [34, 24, 20], [30, 5, 3], [17, 9, 7], [9, 0, 0], [17, 8, 7], [162, 22, 10], [23, 9, 7], [31, 8, 1], [22, 7, 4]]


In [15]:
user_review = {
    0:'Negative',
    1:'Positive',
    2:'Neutral'
}

In [16]:
X = dataset.iloc[:,1:2].values

In [17]:
print(X.shape,labels.shape)

(2000, 1) (2000, 1)


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,WordNetLemmatizer
from sklearn.metrics import accuracy_score

In [19]:
def textPreprocessing(dataset):
    stopwords_list = stopwords.words('english')
    stopwords_list.extend([',','.','-','!','@'])
    wnet = WordNetLemmatizer()
    tfidf = TfidfVectorizer()
    messageList = []
    for i in dataset.index:
        tokens = word_tokenize(dataset['Translated_Review'][i])
        temp = []
        for token in tokens:
            if token.lower() not in stopwords_list:
                temp.append(token.lower())
        for i in range(len(temp)):
            temp[i] = wnet.lemmatize(temp[i],pos='v')
        msg = ' '.join(temp)
        messageList.append(msg)
    vect_array = tfidf.fit_transform(messageList)
    return vect_array.toarray(),tfidf

In [20]:
vect_array,tfidf = textPreprocessing(dataset)

In [21]:
X_train,X_test,y_train,y_test = train_test_split(vect_array,labels,test_size=0.25)

In [28]:
svm = SVC(kernel='linear')
svm.fit(X_train,y_train)

C:\Users\Akhilesh Kr. Pandey\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [29]:
y_pred = svm.predict(X_test)

In [30]:
print(accuracy_score(y_pred,y_test))

0.828


In [37]:
def review_dataset(review):
    df = pd.DataFrame()
    df['Translated_Review'] = [review]
    return df

In [38]:
print(review_dataset('Works great especially going grocery store'))

                            Translated_Review
0  Works great especially going grocery store


In [49]:
vect = tfidf.transform(['Mortal Kombat 11 delivers the best fighting gameplay in the franchise to date, with a healthy dose of style.'])

In [50]:
print(vect.toarray())

[[0. 0. 0. ... 0. 0. 0.]]


In [51]:
pred = svm.predict(vect.toarray())

In [52]:
print(user_review[pred[0]])

Positive
